<h1 style="text-align: center;">Open AI API</h1>

In this notebook, we used `Open AI API` with few-shot learning to find the classification results of treating sentences as a binary classification to highlight them.
# Libraries

In [1]:
# Connecting to Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
colab_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', colab_path)
sys.path.insert(0,colab_path)

Mounted at /content/drive


In [2]:
! pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


In [3]:
pip install langchain --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.2 MB/s eta 0:00:00


In [4]:
!pip -q install langchain openai tiktoken --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.3 MB/s eta 0:00:00


In [5]:
#Uploading the API key .env
from google.colab import files

uploaded = files.upload()

Saving OpenAI-API.env to OpenAI-API.env


In [6]:
#check the key
!ls

drive  notebooks  OpenAI-API.env  sample_data


In [7]:
!pip install python-dotenv --quiet

In [8]:
import os
from dotenv import load_dotenv

load_dotenv('OpenAI-API.env')

api_key = os.getenv("OPENAI_API_KEY")
print("API Key loaded:", api_key is not None)

API Key loaded: True


In [9]:
!pip show langchain

Name: langchain
Version: 0.2.11
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [10]:
!pip install langchain-community langchain-core --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.4 MB/s eta 0:00:00


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.prompts import ChatPromptTemplate

In [12]:
llm = ChatOpenAI(temperature=0, model="gpt-4o")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
!pip install -qU langchain-openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00


# LangChain
The defined schema (fact extraction in the context of each research paper) for assigning a label to a sentence/part of a sentence (chunk) as a human, has been summarized and used as the description for our chat.
As a part of a sentence might be worth highlighting and not all of it, we assigned 1 to the whole sentence indicating that the sentence is worth highlighting.
## Test 1
In our first attempt, we used 2 research papers, the first one for defining some examples to the model and the other for testing.

In [14]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

class Classification(BaseModel):
    highlight: int = Field(
        ...,
        description="describes if a sentence is worth extracting based on having any of the following feature: 1. Core Concepts\
        2. Technical specifications and methodologies 3. Key concepts and hypotheses 4. Underlying mechanisms and reasons for observed results\
        5. Practical applications 6. Specific metrics used in research 7. Comparison with previous research and advantages/improvements of the current research\
        8. Integration with other fields and combined functionalities 9. Quantitative/qualitative results of the project\
        10. Outcomes, analysis and explanations of figures and charts 11. Main results and conclusions",
        enum=[0, 1],
    )

# few-shot examples
few_shot_examples = [
    {
        "input": "Here we show that woodpile (WP)-structured Ir, consisting of 3D-printed, highly-ordered Ir nanowire building blocks, improve OER mass activity markedly.",
        "output": 1
    },
    {
        "input": "there is no difference in the average oxidation state of iridium oxide on the surface of the catalysts.",
        "output": 1
    },
    {
        "input": "In contrast to the signiﬁcant degradation of the active surface area of the nanoparticle-type catalysts, the WP samples have more capability to sustain an active surface area.",
        "output": 1
    },
    {
        "input": "The electrochromic phenomenon in WO3 occurs with the intercala-tion of M+ cations such as H+, Li+, and Na+ from the electrolyte and the reaction mechanism is given as",
        "output": 0
    },
    {
        "input": "The ﬁlm thickness determined by a Veeco Dektak 150 surface proﬁlometry instrument was 165 ± 10 and 300 ± 25 nm for ﬁlms coated on Ni foam and ITO substrates, respectively.",
        "output": 0
    },
    {
        "input": "X-ray photoelectron spectroscopy (XPS) studies were carried out on a PHI Quantera II spectrometer with Al Kα radiation with the energy of 1453 eV.",
        "output": 0
    },
]

# convert examples to prompt
few_shot_prompts = "\n".join(
    f"Input: {example['input']}\nOutput: {example['output']}\n" for example in few_shot_examples
)

#modify the prompts
tagging_prompt = ChatPromptTemplate.from_template(
    f"""
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Here are some examples:
{few_shot_prompts}

Now, process the following passage:

Passage:
{{input}}
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-4o")


chain = LLMChain(prompt=tagging_prompt, llm=llm)


def process_output(output):
    highlight_value = int(output.strip().split(": ")[-1])
    return Classification(highlight=highlight_value)



/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
#Sample test - sentences with label 0
doc1_unhighlight=[
    "Despite highly promising characteristics of three-dimensionally (3D) nanostructured catalysts for the oxygen evolution reaction (OER) in polymer electrolyte membrane water electrolyzers (PEMWEs), universal design rules for maximizing their performance have not been explored.",
    "The 3D nanostructuring-based improvement of ECSA and ECSA-speciﬁc activity enables our well-controlled geometry to afford a 30-fold higher mass activity of the OER catalyst when used in a single-cell PEMWE than conventional nanoparticle-based catalysts.",
    "Among various candidates, Ir-based oxides have been the most widely investigated as a potential catalyst material with high catalytic activity and moderate stability.",
    "The generation of practical-level current density (>1–2Acm−2) typically requires a high loading (0.5–3mgcm−2) of Ir with a cost that is almost twice that of Pt",
    "Therefore, it is highly desirable to develop an innovative technology to raise the mass activity of Ir-based OER catalysts to the targeted level.",
    "also can be designed to be beneﬁcial for gas delivery due to the existence of more efﬁcient gas-transporting channels compared to random aggregates of nanoparticles. In previous research, random or vertically aligned nanowires (NWs) have been studied, but to date the inter-catalyst space, where gas bubbles can be transmitted, has not been systematically engineered.",
    "as shown with the enhanced OER activity achieved by facile bubble detachment via transition from the Cassie state to the Wenzel state wetting of bubbles on 2-dimensionally patterned catalytic surfaces.",
    "For this purpose, intercatalyst space should secure the maximized utilization efﬁciency of the surface area, which is not easy to accomplish with Ir black nanoparticles (one of the state-of-the-art OER catalysts).",
    "Furthermore, ionomers, which are typically mixed with Ir nanoparticles for enhancing ionic conductivity in OER electrodes, may reduce the direct contact between the catalytic surface and reactants and also interfere with efﬁcient transport of the generated gas",
    "Taking all of these factors into account, to maximize catalytic performance, we hypothesize that an",
    "Although these requirements can be partially satisﬁed by the above-mentioned catalyst nanostructures, where the water electrolyte can contribute to ionic conductance over a short distance even in the absence of ionomers, it is difﬁcult to ﬁnd a prior art that demonstrates extensive 3D geometric controllability to deduce practical and universal design rules for highperformance OER catalysts.",
    "The WP structure is attained by systematic engineering of NW building blocks and 3D geometry, which is carried out via ultrahighresolution nanotransfer printing.",
    "which is supported by the extensive control of 3D geometry and the scanning electrochemical microscopy (SECM) analysis data combined with microﬂuidic calculations.",
    "Without any alteration of crystallographic structures or oxidation states, WP-structured Ir catalysts achieve a high mass activity value of 140 A/mg (at 1.8 V) in a single-cell PEMWE, which is 30 times higher than that of the state-of-the-art commercial Ir nanoparticle catalysts (Ir black).",
    "More detailed principles of S-nTP are described in our previous paper.",
    "With the S-nTP process, highly ordered NW arrays can be printed on a macroscopic area with controlled size and orientation of NWs. WP-structured Ir was designed to contain well-deﬁned macropores (>200 nm) at the inter-wire space with linkage to the inter-layer mesopores (<50 nm).",
    "Figure 1c–e presents the basic building blocks (P1200, P400, and P200; unit of the numbers = nm) of Ir NWs labeled after the periodicity of the NW arrays.",
    "Furthermore, the 3D geometry can be tuned using the same building blocks.",
    "For example, NWs having a WP structure can be stacked perpendicularly into two layers (Fig. 1f) and ten layers (Fig. 1g, Supplementary Fig. 2a, b), or a parallel woodpile (PWP) structure (Fig. 1h, Supplementary Fig. 2c, d).",
    "The extensive controllability in the structure of highly ordered NW array stacks was expected to contribute to clariﬁcation of the key factors in enhancing the mass activity of nanostructured catalysts.",
    "In order to investigate whether there is any critical change in the fundamental properties of Ir caused by the S-nTP process, we compared the physiochemical characteristics of 3D WP catalysts with those of Ir black nanoparticles (Premetek)—a commercial Ir catalyst for OER.",
    "As the physiochemical characterization in Supplementary Figs. 3 and 4 showed",
    "we used the term—S-nTP NWs for indicating all the Ir NWs fabricated with S-nTP method.",
    "Supplementary Fig. 3), which is in line with the selected-area electron diffraction (SAED) patterns (insets of Fig. 2a, c).",
    "The peak broadening for the S-nTP NWs and the Ir black can be explained by the small grain size (~3–5 nm in diameter, Fig. 2b) in the NW building blocks and the small particle size (3.4 nm in diameter, Fig. 2c) of the Ir black catalysts.",
    "The Ir 4f XPS peaks shown in Fig. 2e, which can be deconvoluted into Ir 4f7/2 and Ir 4f5/2 binding peaks.",
    "In both cases, the two types of iridium oxide peaks comprise ~32 and 36% of the overall Ir 4f peak, and the metallic Ir appearing at 60.9 eV28 occupies 32% (Supplementary Table 3).",
    "Similarly, the O 1s XPS spectra (Supplementary Fig. 5) and the ratio of IrO peak to IrOH peak (Supplementary Table 2) were analogous for both samples.",
    "The Ir-LIII absorption edge, which probes 2d to 5d electronic transitions, is sensitive to changes in the oxidation state of Ir.",
    "The XANES spectra (Supplementary Fig. 6) showed no shift of the absorption edge energy among the samples,",
    "As the few-atom layer of the Ir metallic surface partly becomes iridium oxide in the air, the formation of iridium hydroxo species (*OH) on the surface can elongate the average Ir–O bonding in the sampels.",
    "Fig. 1 Fabrication of 3D Woodpile-structured Ir catalysts. a Illustration of woodpile-structured Ir thin ﬁlm containing mesopores and macropores for OER",
    "b Fabrication process of woodpile-structured Ir with repeating solvent-assisted nanotransfer printing (S-nTP).",
    "Building blocks of woodpile structure named according to the periodicity of the nanowire arrays having different line width and period: scale bar in (c)is2μm, and scale bars in (d) and (e) are 1 μm.",
    "The loaded amount of Ir was controlled quantitatively along the number of stacking layers. f 2- and g 10-layer perpendicular-stacking of the P1200 buildingblock. h Parallel stacking of P1200 building block. Scale bars in (f–h) indicates 5 μm.",
    "Fig. 2 Characteristics of building blocks of Ir woodpile structure. a TEM image (inset: corresponding SAED pattern with 5 1/nm scale bar) and b HRTEM image of Ir NWs fabricated by S-nTP. Scale bars in (a) and (b) are 100 and 10 nm, respectively",
    "c TEM image (200 nm scale bar), HRTEM image (20 nm scale bar), and SAED patterns (5 1/nm scale bar) of Ir black nanoparticles.",
    "d Comparison of the XRD patterns of printed NWs and Ir black nanoparticles, showing the same face-centered cubic (FCC) crystal structure.",
    " e XPS and f non-phase corrected Fourier transform of k2-weight EXAFS data, demonstrating similar oxidation states of Ir in both samples.",
    "Solely at the ﬁrst cycle (Fig. 3d) of the cyclic voltammetry (CV) in Supple-mentary Fig. 7, it was possible to calculate the ECSA from the hydrogen underpotential deposition (HUPD) peak.",
    " which was calculated on the basis of the cross-sectional image of the NWs presented in Supplementary Fig. 1. ",
    "The Ir utilization efﬁciency (94–98%, ratio of measured ECSA to CSA, Supplementary Table 1) of the samples fabricated by S-nTP was substantially higher than the utilization efﬁciency of the Ir black sample (47.7%, Supplementary Table 5).",
    "Whereas the CSA of Ir black was at least 160% larger than that of WP samples, the ECSAs of the two types of samples were comparable.",
    "The ECSA of the Ir black sample was measured to be at least 26%larger than those of the WP samples, where the ECSAs varied depending on their geometries.",
    "The mass activities of the samples, as shown in Supplementary Table 7, were estimated with the current densities at 1.55 V in the linear sweep voltammetry (LSV) curves (measured at 5 m V s−1 scan rate with iR compensation, Fig. 3b and Supplementary Fig. 11a) and their normalization based on the Ir loaded amount in each sample.",
    "The highest mass activity among the 3D WP-structured samples was obtained for the WP400 with values of 0:513 A mg􀀁@11:5VRHE,1:92A mg􀀁@11:53 VRHE , and 3:76A.",
    "These correspond to 4.8 times the mass activity of Ir black􀀁0:79 A mg􀀁@11:55 VRHE (Fig. 3a) and are comparable to the value 0:38 A mg􀀁@ and 0:7A mg􀀁1 12 from recent studies, while these studies did not demonstrate device applications for water electrolysis. ",
    "In terms of reducing overpotential, although some studies demonstrated a low overpotential of 248 mV for the generation of 10 mA cm−2, which is 8.2% smaller than that (270 mV overpotential) of this study",
    "To elucidate this phenomenon, the ECSA-speciﬁc activity (A m􀀁2ECSA) should be taken into account (see Fig. 3e and Supplementary Fig. 12)",
    " The ECSA-speciﬁc activity was obtained by dividing the mass activity (at E = 1.55 VRHE) by the ECSA. In Fig. 3e and Supplementary Fig. 12,"
    " which will be discussed in detail in the next section,"
    " We attempted to elucidate the effect of 3D nanostructuring on the ECSA-speciﬁc activity.",
    " As shown in Fig. 3e, the ECSA-speciﬁc activity of 3D WP P1200, P400, and P200 samples was 6.2-, 6.0-, and 4.6-fold higher than that of Ir black at 1.55 VRHE, respectively.",
    "The more than twofold improvement of the mass activity compared to PWP structures (Fig. 3a)",
    "As illustrated in Fig. 4a, for general OER catalysts including commercial random nanoparticles and our 3D-nanostructured catalysts, it is expected that there are more factors at work beyond simple bubble detachment from catalytic surfaces: bubble coalescence and removal from the catalyst layer.",
    "Fig. 3 Comparison of OER catalytic half-cell performance.",
    "a The mass activity of 4-layered 3D WP Ir and Ir black samples, showing a 4.8 times higher mass activity of the WP400 catalyst compared to Ir black.",
    "Parallel stacking of NWs resulted in a substantial decrease of mass activity.",
    "b Linear sweep voltammetry of 4-layered WP Ir and Ir black. The inset presents the Tafel plots for the Ir catalysts, showing analogous Tafel slopes.",
    " c The ECSA evaluated by the HUPD peak in (d) the cyclic voltammetry range of 0.05–1.4 VRHE, which oxidizes Ir surfaces electrochemically.",
    "e ECSA-speciﬁc activity calculated by dividing the mass activity by the ECSA.",
    "The error bars in (c) and (e) indicates the standard deviation of measurements.",
    "f Normalized current density for 500 cycles of repeated chronoamperometry for OER stability test.",
    "The inset shows the ratio of integrated surface charge values before and after 500 cycles, presenting a similar trend as the current density.",
    "Fig. 4 Facile transport of evolved O2 gas in the WP catalyst layer.",
    "a Schematic illustration of bubble generation and removal mechanism within the 3D-nanostructured electrocatalyst.",
    "b FFT amplitude spectra of currents recorded at the SECM tip (potential of the sample: 1.4 VAg/AgC1).",
    "Note that the higher frequency of the peak indicates a faster cycling of formation and detachment of O2 bubbles.",
    "c Dynamic light scattering analysis spectra, showing the size distribution of O2 bubbles generated by the WP and PWP catalysts, respectively.",
    "Comparison of mass activity depending on the stacking direction (perpendicular vs. parallel) of d P200, e P400, and f P1200.",
    "Insets in (d, e) shows the narrower range of x- and y- axis than the entire plots for expansion for expanded view of the comparison of mass activity near 1.55 VRHE.",
    "During the SECM measurement, current ﬂuctuations (Supplementary Fig. 13b, c) were monitored by positioning a microelectrode in the proximity of the sample surface, which was biased electrically to evolve gas bubbles.",
    "Extraction of the characteristic frequency of the peaks in current is enabled by fast Fourier transformation (FFT, Fig. 4b),",
    "The higher frequency of the WP nanostructure (8.65 and 11.8 Hz) implies faster formation and removal of O2 bubbles during the OER, compared to the PWP nanostructure (4.35 and 5.48 Hz).",
    "Using microﬂuidic calculations, it is possible to estimate the ratios of theoretical frequencies of bubble removal based on the buoyancy effect (driving force for bubbles to escape from the catalyst layer) and the bubble-catalyst interface surface tension (wetting condition).",
    " If the evolved gas bubbles are much smaller than the pore size, V1=3 􀀁 pg , we can treat the bubbles and their transport as simple particles transporting through a porous media, using Darcy’s law",
    "Here, V is the volume of gas and pg is the pore size of the porous structure. We can then conclude that the open and ordered WP1200 structure has almost a ﬁve-time larger mass transport rate than the PWP 1200 structure (see Supplementary Note 1).",
    "On the other hand, if evolved gas bubbles have similar or larger size than that of the pores, we should consider the force balance between the buoyancy effect (Fb) and the bubble-catalyst interface surface tension force (Fs), Fb (=ΔρVg)~ Fs ð¼ γ‘Þ, where Δρ is the density difference between gas and liquid, g is the gravitational acceleration, γ is the surface tension, and ‘ is the perimeter length for the gas–liquid wetting.",
    "We compare a critical volume to be detached from the different nanostructures.",
    "for instance the surface adhesion force of the parallel NWs is much larger than that of the crossing NWs.",
    "Thus, the critical volume for the, PWP geometry is alw ys larger than the WP structure case, i.e., WP ¼􀀁8Δpρgg<􀀂VPWP ¼􀀁17Δpρg 􀀂(see the Supplementary Information).",
    "The frequency of bubble removal from the catalyst layer (fb) can be scaled from the inertia capillary time scale τcap 􀀂 fb􀀂 ΔρﬃﬃﬃﬃﬃﬃﬃﬃﬃﬃﬃﬃﬃV=γﬃ.",
    "Thus, the frequency ratio of bubble removal can be fb;WP fb;PWP ¼ ffiffiffiffi 17 8 q 1:5, where the volume was considered from the critical volume for each structure (see details in Supplementary Note2).",
    "We characterized and conﬁrmed the larger bubbles that must be formed by the coalescence within the 3D catalyst layer via a dynamic light scattering (DLS, Fig. 4c) analysis on the electrolyte, where we conducted LSV once with different 3D geometries (WP 1200 and PWP1200).",
    "The larger bubble size (2838 nm) from the PWP1200 structure was estimated to be 19% larger than that (2381 nm) of the bubbles from the more open and ordered WP1200 samples.",
    "This can be attributed to the more facile escape of the larger bubbles in the WP structure before growing into much larger ones, which is consistent with the results of the SECM analysis and theoretical calculations. ",
    "engineering 3D nanostructures can accelerate the rate of OER per unit ECSA by facilitating the removal of the generated gas bubbles.",
    "There are further experimental results supporting that the ECSA-speciﬁc activities of the 3D WP catalysts are governed by the eases of gas bubble transport, which is controlled by the geometric parameters of the 3D catalysts.",
    "which were maintained within a deviation of only 4.6%(Fig. 3c).",
    "The reduction in the ECSA-speciﬁc activities resulted in a noticeably lower mass activity of PWP than that of the WP-structured catalysts (Fig. 3a and Supplementary Fig. 11b).",
    " The opening ratio (OR) of 3D catalysts (ratio of the spacing between NWs divided by NW width), which is in the order of P200 (OR = 0.250) > P1200 (OR = 0.208) > P400 (OR = 0.125), is thought to be inversely scaled with the ease of bubble transport.",
    "This is consistent with the considerable loss of ECSA-speciﬁc activity by the structural change from WP to PWP in Fig. 3e (indicated with dotted red arrows).",
    "For example, the change of building block from P200 to P400 resulted in a 31% increase of ECSA-speciﬁc activity, while there was only a 3% improvement by the change from P400 to P1200.",
    "which can be characterized by the Ohmic loss measured in the EIS analysis at the OER-occurring potential (1.55 V vs RHE) in Supplementary Fig. 10.",
    "Regardless of the stacking directions, all NW stacks showed a comparable Ohmic resistance to that of the Ir black samples (Supplementary Table 6).",
    "Figure 3f presents normalized current densities over the course of 500 repeating chronoamperometry for the three types of 4-layered 3D WP samples and an Ir black nanoparticle electrode as a reference.",
    "The normalized current densities (>0.8) after 500 cycles of the 3D WP samples are comparable to each other, but for the Ir black nanoparticles, the current density decreased by ~40%.",
    "The trend of the superior stability of the WP samples was maintained for even more (1000) cycles of the OER (Supplementary Fig. 16a), which was also demonstrated with chronopotentiometry in Supplementary Fig. 17.",
    "To quantify ECSA, as an alternative for the estimation based on HUPD peaks, we exploited electric double-layer charge calculated from the CV data (Supplementary Fig. 8) and capacitances (Supplementary Fig. 9h) measured with another CV ranging 1.1–1.2 VRHE at various scan rates (Supple-mentary Fig. 9a–g).",
    " It was reported that coalescence or detachment of Ir nanoparticles could lead to a severe loss of surface area (>50%).",
    "as shown in Supplementary Fig. 18, although they experienced similar amorphization and hydrated oxidation (Supplementary Fig. 16d–g) of the Ir-based OER catalysts in the acidic electrolyte.",
    "This structural robustness suggests that the inherent dissolution of Ir is the main factor underlying the gradual OER performance degradation of the WP structure. However, as shown in Supplementary Fig.",
    "Previous studies reported that there is an apparent tendency of decelerated Ir dissolution in properly conﬁned nanostructures.",
    "Also, for our WP samples, the 4-layer 3D geometry having more conﬁned conditions improved the OER stability relative to the 2-layer structure (Supplementary Fig. 16b), suggesting that",
    "We next demonstrate the outstanding catalytic performance of WP structures when applied to single-cell PEMWE devices.",
    "The current density of the single cell (Fig. 5a) containing WP1200 as the anodic catalyst increased as the number of stacking layers was raised from 10 layers to 30 layers, as shown in the I–V curve of Fig. 5c.",
    "This can be understood by the linearly enlarging surface area along the number of stacking layers, as conﬁrmed in the half-cell test (Supplementary Figs. 14a and 15).",
    "The same ten-dency was observed for WP400 and WP200 samples (Supple-mentary Fig. 20).",
    "Among the 30-layered WP Ir samples, the current densities were in the order of P1200 > P400 > P200 (Fig. 5d).",
    "The 30-layered P1200 sample recorded a maximum current density of 5.2 A cm−2 at 2.0 V, which is more than double of the conventionally reported maximum current densities of 1.0–3.0 A cm−2 at >1.8 V.",
    "In our setting of single-cell measurements, the WP 1200 30-layered sample showed the maximum current density with an Ir loading of ~45 μgcm−2, and the maximum current density was 4.3-times higher than that of the cell based on Ir black nanoparticles (Fig. 5d) with comparable Ir black loading (50 μg cm−2).",
    "In addition, the PEMWE equipped with 30-layer WP1200 sustained the advanced performance for longer than 45 h as shown in Supplementary Fig. 21.",
    "Moreover, the best membrane electrode assembly (MEA) based on 10-layer P400 catalysts recorded a mass activity (40.6 A/mg) at 1.6 V, which corresponds to 25 times and 17 times the values of the Ir-black-based cells with a comparable Ir loading amount (50 μgcm−2) and with the optimum Ir loading (=100 μgcm−2) measured in this study (Fig. 5e).",
    " At 1.8 V, even 30 and 21 times higher enhancement of the mass activity (vs. Ir black nanoparticles having comparable loading and optimized mass activity, respectively) was achieved (Fig. 5f)",
    " Among the 10-layered WP catalysts, the mass activity was in the order of WP400 > WP120 > WP200, which is consistent with the half-cell test performed with 4-layered samples.",
    "(Fig. 3a) A different trend, however, was observed for the larger number of stacking layers: WP1200 > WP400 > WP200 for 20- and 30-layered WP structures.",
    "The decrease in mass activity accompanied with the addition of more layers was much less signiﬁcant for P1200 compared to P400 and P200.",
    "In the cases of P400 and P200, the mass activity decreased by ~50% from 10 layers to 20 layers and 10–15% from 20 layers to 30 layers at both 1.6 and 1.8 V.",
    "In contrast, P1200 showed only a 10–20% decrease of the mass activity upon changing from 10 layers to 20 layers and with additional stacking from 20 to 30 layers.",
    " As the path length for transport of O2 would be increased for a higher number of stacked layers, while increasing amount of bubbles, the decrease in mass activity accompanied with the addition of layers was much less signiﬁcant for P1200 having larger channel size compared to P400 and P200.",
    "providing practical design rules for more efﬁcient OER catalysts.",
    "We expect that these outcomes will provide a large step toward the development of highly feasible OER catalysts by simply controlling their 3D microstructures even without any change of physiochemical characteristics.",
    "Therefore, there will be abundant room for further improving the OER performance of our WP-structured catalysts via synergistic combination with other approaches based on new elemental compositions."
]

In [ ]:
#Sample test - sentences with label 1
doc1 =[
"Here we show that woodpile (WP)-structured Ir, consisting of 3D-printed, highly-ordered Ir nanowire building blocks, improve OER mass activity markedly.",
"The WP structure secures the electrochemically active surface area (ECSA) through enhanced utilization efﬁ-ciency of the extended surface area of 3D WP catalysts.",
"systematic control of the 3D geometry combined with theoretical calculations and various electrochemical analyses reveals that facile transport of evolved O2 gas bubbles is an important contributor to the improved ECSA-speciﬁc activity.",
"The 3D nanostructuring-based improvement of ECSA and ECSA-speciﬁc activity enables our well-controlled geometry to afford a 30-fold higher mass activity of the OER catalyst when used in a single-cell PEMWE than conventional nanoparticle-based catalysts.",
"Water electrolysis is a promising method to produce H2, a clean energy source, potentially serving as a central technology in the H2-based renewable energy cycle.",
"polymer electrolyte membrane water electrolyzers (PEMWEs) are attracting strong interest due to their compact-ness, superior energy efﬁciency, excellent purity of produced hydrogen, and high production rate.",
"surface area per loading amount of catalytic material, (2) well-deﬁned pore structures for highly efﬁcient transport of evolved gas species, (3) long-range connectivity of building blocks for high electronic conductivity, and (4) capability of ionomer-free operation.",
"we report that 3D-geometry-controlled Ir catalysts with a woodpile (WP) structure suggest a new direction for the OER electrocatalysts to improve the PEMWE performance.",
"The extensive controllability of the fabrication method enabled control of the NW-to-NW spacing and layer-to-layer alignment angles in the WP nanos-tructure.",
"we reveal that the simultaneous improvements of ECSA and ECSA-speciﬁc activity result from uniform access of water reactants to the catalytic surface and easy transport and removal of evolved oxygen gas products through the well-deﬁned pore channels",
"We fabricated 3D WP-structured Ir electrocatalysts with stacking 1D NW arrays processed by solvent-assisted nano-transfer printing (S-nTP)",
"negligible differences among the three types of catalysts built with the building blocks of P1200, P400, and P200",
"First, broad X-ray diffraction (XRD) peaks were observed for both the S-nTP Ir NWs and nanoparticle-type Ir black samples",
"The diffraction patterns of the samples reveal that both samples are mainly composed of polycrystalline Ir metals having a face-centered cubic (FCC) crystal structure.",
"The chemical nature of the surface species of the as-prepared Ir samples was also examined by X-ray photoelectron spectroscopy (XPS).",
"there is no noticeable difference between Ir black and S-nTP NWs.",
"For a more in-depth evaluation of the oxidation state and the local structural environment of Ir, the WP samples composed of S-nTP NWs and nanoparticle-type Ir black reference were further investigated by X-ray absorption near edge structure (XANES) spectroscopy and extended X-ray absorption ﬁne structure (EXAFS) spectroscopy for the Ir-LIII edge."
"there is no difference in the average oxidation state of iridium oxide on the surface of the catalysts.",
"For characterizing the local coordination environment of Ir, a two-cluster model was employed to ﬁt the EXAFS data, revealing elongation of the Ir–O bonding distance due to Ir oxide formation on the surface for both Ir black and S-nTP NWs.",
"XPS and XAS analysis data suggest the existence of Ir oxide on the metallic Ir cores, which were identiﬁed from the XRD and transmission electron microscopy (TEM) analyses, conﬁrming that the WP samples do not have a prominent difference in oxidation state compared to the Ir black particles.",
"To elucidate the effect of 3D nanostructuring on the OER performance, we measured the ECSA (ECSA per loaded mass of Ir) of the samples.",
"The ECSAs of the three types of 4-layered Ir WP nanos-tructures were compared to the calculated speciﬁc surface area (CSA)",
"our WP catalyst achieved 187 times higher mass activity at 1.55 VRHE.",
"Consideration of the ECSA (Fig. 3c) of the 3D WP catalyst comparable to the Ir black nanoparticles suggests that the ECSA alone does not explain the 4.8-fold higher mass activity, and thus an additional factor should be taken into account.",
"all three types of WP-structured samples show superior ECSA-speciﬁc activity compared to the nanoparticle-type Ir black sample",
"there is no difference in the physiochemical characteristics related to the catalytic activity determined from XPS, XRD, XAS, and TEM analyses and corresponding Tafel slopes",
"A possible explanation for the superior ECSA-speciﬁc activity in the 3D WP-structured catalysts is efﬁcient removal of oxygen bubbles from the catalyst layer.",
"despite the use of identical NW building blocks suggests that the open and ordered characteristics of the 3D WP structures can lead to accelerated transport and removal of gas bubbles from the catalyst layer.",
"two hypotheses were set based on the electrochemical analysis data: (A) the open and ordered 3D structure shortens the residence time of bubbles in the internal space of catalyst layer, and (B) longer residence time results in more severe coalescence of bubbles in the ﬂuid.",
"The ﬁrst hypothesis (A) was investigated experimentally with SECM and theoretically with microﬂuidic calculations.",
"We conducted SECM characterizations to compare WP P1200, having the highest speciﬁc activity among the samples, with PWP 1200.",
"At the narrow space, the formation of insulating gas bubbles leads to a decrease in the current density.",
"As the electrical bias continues, the detachment of the insulating gas bubbles increases the current density again, appearing as a current peak.",
"the multiple peaks could be caused by two bubbles evolved from a catalytic site or another activated site at a typical potential range.",
"the bubbles’ surface adhesion force can vary depending on the wire alignment.",
"The second theoretical model calculated under the assumption of bubble size similar to or larger than pore channel size in the catalyst is consistent with the results of the SECM measurement data.",
"This implies that the bubbles having a size similar to or larger than that of the pores determines the frequency of bubble formation and removal from the catalyst layer.",
"the bigger bubbles from PWP tend to have a larger size because of the longer residence time in the catalyst layer",
"in the DLS spectra, it can be seen that relative intensity of the bigger bubbles from the PWP structure is relatively lower than that of the smaller bubbles, while the tendency is reversed in the case of bubbles from the WP structure.",
"coalescence of bubbles, which is driven by the reduction of the overall surface energy, should be prevented for higher OER activity.",
"the coalesced gas bubbles within the catalyst layer hinder the approach of the reactant water to the catalytic surface.",
"When the stacking direction of three types of NW building block arrays were changed from perpendicular (WP) to parallel (PWP), the impact of decrease in the ECSA-speciﬁc activity on the mass activity overwhelmed the that of ECSAs",
"the decrease in mass activities resulting from the change of stacking direction of Ir NW arrays for the denser structures became relatively less signiﬁcant than that for the more open NW building blocks.",
"the ECSA-speciﬁc activity increases less rapidly with increasing macropore size in the 3D WP samples, suggesting that there is an optimal macropore size.",
"The ECSA-speciﬁc activity can be affected not only by the mass transport of the reactants/products of the reaction within the nanostructures but also by the conductivity of the electrons/protons",
"in the 3D WP structures, the water inside the well-deﬁned nanoscale pores can provide ionic conductance over a short distance even in the absence of ionomers",
"The decrease in the integrated surface charge during the 500 cycles can be attributed as the dominant factor in the OER stability of the samples.",
"In contrast to the signiﬁcant degradation of the active surface area of the nanoparticle-type catalysts, the WP samples have more capability to sustain an active surface area.",
"Agglomeration of Ir nanoparticles also could be expedited by ionomer degradation accelerated by the applied voltage",
"the WP geometry was almost perfectly maintained even after 500 cycles of the degradation test",
"Ir dissolution rate was relatively smaller for the WP samples compared to Ir black.",
"optimization of the 3D geometry may be able to provide a simultaneous inﬂuence on OER activity and stability.",
"The mass activity is approximately four times greater than the highest previously reported value among those of Ir-based catalysts",
"the 3D WP structures allow more efﬁcient transport of O2 gas bubbles due to their far more open architecture",
"The correlation between the geometric properties of WP structures and the mass activity further validates the critical role of bubble transport in the catalyst.",
"These results indicate that well-deﬁned macropore channels with appropriate size WP structures are critical for high-current density and mass activity, although the optimum structural parameters depend on the operation conditions and the total loading amount of Ir.",
"WP-structured Ir catalysts were designed and successfully fabricated via a high-resolution printing process to enable extremely high mass activity in the OER, which is driven by both high ECSA and ECSA-speciﬁc activity.",
"A half-cell test demonstrates that the WP catalyst generated 4.8-times higher mass activity than the Ir black nanoparticles despite no difference in the physiochemical properties.",
"when applied to a single-cell PEMWE device, the new catalyst achieved far more enhanced mass activity, up to 30 times that of Ir black.",
"Systematic tuning of the 3D geometry and NW building blocks revealed that O2 bubble transport, which was characterized with SECM, is a key parameter in improving the performance of the OER catalysts.",
        ]

In [ ]:
import pandas as pd

results = []


for text in doc1_unhighlight:
    model_output = chain.run({"input": text})
    classification_result = process_output(model_output)
    results.append({"Text": text, "Output": classification_result.highlight, "Label": 0})

for text in doc1:
    model_output = chain.run({"input": text})
    classification_result = process_output(model_output)
    results.append({"Text": text, "Output": classification_result.highlight, "Label": 1})


results_df = pd.DataFrame(results) #save the result list as dataframe

# accuracy, precision, recall, and F1 score
true_labels = results_df["Label"]
predicted_labels = results_df["Output"]

accuracy = (true_labels == predicted_labels).mean()


true_positive = ((true_labels == 1) & (predicted_labels == 1)).sum()
false_positive = ((true_labels == 0) & (predicted_labels == 1)).sum()
false_negative = ((true_labels == 1) & (predicted_labels == 0)).sum()

precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

# Save dataframe as .CSV
results_df.to_csv('/content/drive/My Drive/Colab Notebooks/API_result.csv', index=False)


Accuracy: 0.5611111111111111
Precision: 0.40384615384615385
Recall: 0.711864406779661
F1 Score: 0.5153374233128835


In [ ]:
results_df

,Text,Output,Label
0,Despite highly promising characteristics of th...,0,0
1,The 3D nanostructuring-based improvement of EC...,1,0
2,"Among various candidates, Ir-based oxides have...",1,0
3,The generation of practical-level current dens...,0,0
4,"Therefore, it is highly desirable to develop a...",0,0
...,...,...,...
175,These results indicate that well-deﬁned macrop...,1,1
176,WP-structured Ir catalysts were designed and s...,1,1
177,A half-cell test demonstrates that the WP cata...,1,1
178,"when applied to a single-cell PEMWE device, th...",1,1


## Test 2
We tested the whole test dataset, used in other notebooks related to treating a sentence as binary classification (Llama3 fine tuning).

In [15]:
import pandas as pd
sentence_df = pd.read_csv('/content/drive/MyDrive/Final Project/Field Project/Highlighted Papers/Sentence_dataframe.csv', index_col=False)

In [16]:
sentence_df.head(2)

,Sentence,File_Name,Parse_Tree,Processed_Tree,Chunks,Highlight
0,Despite highly promising characteristics of th...,cleaned_Fulltext_Highly efficient.txt,(ROOT (S (PP (IN Despite) (NP (NP (ADJP (RB hi...,(S (PP (IN Despite) (NP (NP (ADJP (RB highly) ...,Despite highly promising characteristics of th...,0
1,"Here we show that woodpile -structured Ir, con...",cleaned_Fulltext_Highly efficient.txt,(ROOT (S (ADVP (RB Here)) (NP (PRP we)) (VP (V...,(S (ADVP (RB Here)) (NP (PRP we)) (VP (VBP sho...,"Here we show that woodpile-structured Ir,consi...",1


In [17]:
# create the same test set as Llama3 fine tuning
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(sentence_df, test_size=0.2, random_state=42, shuffle=True, stratify=sentence_df['Highlight'])


In [19]:
sentences = X_test['Sentence'].tolist()
labels = X_test['Highlight'].tolist()

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

results = []

for sentence in sentences:
    model_output = chain.run({"input": sentence})
    classification_result = process_output(model_output)
    results.append(classification_result.highlight)


accuracy = accuracy_score(labels, results)
precision = precision_score(labels, results)
recall = recall_score(labels, results)
f1 = f1_score(labels, results)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Accuracy: 0.5683139534883721
Precision: 0.5454545454545454
Recall: 0.04013377926421405
F1 Score: 0.07476635514018691


In [21]:
results_df_2 = pd.DataFrame(results)


In [37]:
# save as .CSV
results_df_2.to_csv('/content/drive/My Drive/Colab Notebooks/API_result_2.csv', index=False)

The F1 Score was low, so I thought it might be a problem with my classification report. Therefore, I tested it again.

In [39]:
# check predicted labels shape
results_df_2.shape

(688, 1)

In [28]:
# check actual labels shape
len(labels)

688

In [34]:
# Define a new dataframe for retesting
temp_dataframe = pd.DataFrame()
temp_dataframe['label']= X_test['Highlight']
temp_dataframe['predict']= results

In [35]:
temp_dataframe.head()

,label,predict
2487,0,0
1997,0,0
2240,1,0
2221,0,0
1829,1,0


In [36]:
#accuracy, precision, recall, and F1 score
true_labels = temp_dataframe['label']
predicted_labels = temp_dataframe['predict']

accuracy = (true_labels == predicted_labels).mean()

true_positive = ((true_labels == 1) & (predicted_labels == 1)).sum()
false_positive = ((true_labels == 0) & (predicted_labels == 1)).sum()
false_negative = ((true_labels == 1) & (predicted_labels == 0)).sum()

precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

Accuracy: 0.5683139534883721
Precision: 0.5454545454545454
Recall: 0.04013377926421405
F1 Score: 0.07476635514018691


The results were the same and we can assume that good results in the first test were due to the reason that both training and test examples were from the same subject. However, the model could not generalized to all genres.

In [40]:
!pip freeze > requirements.txt